In [2]:
from graphdatascience import GraphDataScience
import pandas as pd

In [3]:
df = pd.read_csv("../5_InstagramFinder/FinalDataSet.csv")
df = df.drop(["Unnamed: 0", 'url', 'artist_url', 'media_long','dimension','gallery_url','bid','artist_bio'],axis = 1)
df.to_csv("neo4j_data.csv")

In [19]:
df

,id,media,artist_name,work_name,work_year,gallery_name,price,img_url,artist_website,artist_ins
0,painting15,painting,Hebru Brantley,"Death Of The Black Fairy, The Great Debate Pt. 5",2020,Corridor Contemporary,125000.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,www.hebrubrantley.com,https://www.instagram.com/hebrubrantley/?hl=en
1,painting37,painting,Miaz Brothers,Maestro,2022,Maddox Gallery,17500.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,www.miazbrothers.com,https://www.instagram.com/miazbrothers/?hl=en
2,painting41,painting,Sandra Chevrier,La cage où le brouillard traverse l’esprit,2017,West Chelsea Contemporary,5000.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,www.sandrachevrier.com,https://www.instagram.com/sandrachevrier/?hl=en
3,painting45,painting,Florian Eymann,BAYC Clueless Sailor,2022,Avant Gallery,11000.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,www.florianeymann.com,https://www.instagram.com/florianeymann/?hl=en
4,painting52,painting,Miaz Brothers,A Promising Young Man,2022,Maddox Gallery,17500.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,www.miazbrothers.com,https://www.instagram.com/miazbrothers/?hl=en
...,...,...,...,...,...,...,...,...,...,...
2522,design1454,design,Ed Ruscha,He Up and Went Downtown - Plate,2020,West Chelsea Contemporary,3144.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,edruscha.com,NaN
2523,design1463,design,Zhoujie Zhang,Brass Bowtie Chair (SQN1-F2A),2014,Gallery ALL,12500.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,www.zhangzhoujie.com,NaN
2524,design1467,design,Jeff Koons,Lips,2012,Almine Rech,728.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,www.jeffkoons.com,https://www.instagram.com/jeffkoons/?hl=en
2525,design1481,design,Pablo Picasso,Madoura Ceramic Pitcher 'Yan Soleil' Ramié 516,1963,Hirth Fine Art,15000.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,www.pablopicasso.org,https://www.instagram.com/pablopicasso_arty/?h...


In [17]:
# Import the driver
from neo4j import GraphDatabase

# Replace with the actual URI, username, and password
AURA_CONNECTION_URI = "neo4j+s://b271b634.databases.neo4j.io"
AURA_USERNAME = "neo4j"
AURA_PASSWORD = "YBafb8qEcEbvBhXPVR96_wrcKMbeKWlVgZ1lwWQxAXE"

# Instantiate the driver
driver = GraphDatabase.driver(
    AURA_CONNECTION_URI,
    auth=(AURA_USERNAME, AURA_PASSWORD)
)

In [18]:
# Import to prettify results
import json

# Import for the JSON helper function
from neo4j.time import DateTime

# Helper function for serializing Neo4j DateTime in JSON dumps
def default(o):
    if isinstance(o, (DateTime)):
        return o.isoformat()

In [20]:
aritst_name_constraint = """
    CREATE CONSTRAINT FOR (artist:Artist) REQUIRE artist.name IS UNIQUE
"""
gallery_name_constraint = """
    CREATE CONSTRAINT FOR (gallery:Gallery) REQUIRE gallery.name IS UNIQUE
"""
work_id_constraint = """
    CREATE CONSTRAINT FOR (work:Artwork) REQUIRE work.id IS UNIQUE
"""


# Create the driver session
with driver.session() as session:
    session.run(aritst_name_constraint).data()
    session.run(gallery_name_constraint).data()
    session.run(work_id_constraint).data()

In [ ]:
load_artist_csv = """
    LOAD CSV
      WITH HEADERS
      FROM 'neo4j_data.csv' AS row
    MERGE (a:Artist {name: row.artist_name, ins: row.artist_ins, website: artist_website})
      ON CREATE SET m.released = toInteger(row.released), m.tagline = row.tagline
    RETURN count(*)
"""

# Create the driver session
with driver.session() as session:
    # Load the CSV file
    session.run(load_movies_csv).data()

In [10]:
df[['artist_name','artist_website']][:20]


,artist_name,artist_website
0,Hebru Brantley,www.hebrubrantley.com
1,Miaz Brothers,www.miazbrothers.com
2,Sandra Chevrier,www.sandrachevrier.com
3,Florian Eymann,www.florianeymann.com
4,Miaz Brothers,www.miazbrothers.com
5,Miaz Brothers,www.miazbrothers.com
6,Miaz Brothers,www.miazbrothers.com
7,Mr. Brainwash,www.mrbrainwash.com
8,Mau Mau,www.mau-mau.co.uk
9,Yaacov Agam,NaN
